In [1]:
import sys
import os
from datetime import datetime
from datetime import datetime
from utils.df_handle import *
import pandas as pd
import numpy as np
import time
print(os.path.dirname(sys.executable))

d:\data_sale\venv\Scripts


In [2]:
datenow = datetime.now().strftime("%Y%m%d")
param_1 = f"'20161130'"
param_2 = f"'20211012'"
param_3 = f"'{datenow}'"

In [3]:
query = f"EXEC [pr_AR_TrackingDebtConfirm]  @Fromdate={param_1}, @Todate={param_3}"
start=time.time()
df1 = get_ms_df(sql=query, parse_dates={"DateOfOrder": {"dayfirst": True}, "DueDate": {"dayfirst": True}})
# df1.to_csv('doanhthutienmat_data1.csv', index = False)
end=time.time()
duration=end-start
print(duration)

45.72133731842041


In [9]:
df1.columns

Index(['BranchID', 'BranchName', 'SupName', 'ASMName', 'RSMName', 'SlsperID',
       'SlsperName', 'Position', 'DebtInCharge', 'PaymentsForm', 'TermsType',
       'OrderDate', 'OrderNbr', 'DateOfOrder', 'DeliveryUnit', 'DeliveryTime',
       'Terms', 'DueDate', 'CustID', 'CustName', 'Channels', 'SubChannel',
       'Territory', 'Streets', 'Ward', 'District', 'State', 'Attn', 'Phone',
       'PhoneCustInvc', 'CountOpeningOrder', 'OpeiningOrderAmt',
       'CountOrdRelease', 'OrdAmtRelease', 'DeliveredOrder',
       'DeliveredOrderAmt', 'CountReturnOrd', 'ReturnOrdAmt', 'ConfirmAmt',
       'ReasonNoPay', 'CountDebtConfirm', 'DebConfirmAmt',
       'CountDebtConfirmRelease', 'DebConfirmAmtRelease'],
      dtype='object')

In [10]:
df1[checkdup(df1, 2, ['BranchID','OrderDate','OrderNbr','DateOfOrder','DueDate'])]

,BranchID,BranchName,SupName,ASMName,RSMName,SlsperID,SlsperName,Position,DebtInCharge,PaymentsForm,...,DeliveredOrder,DeliveredOrderAmt,CountReturnOrd,ReturnOrdAmt,ConfirmAmt,ReasonNoPay,CountDebtConfirm,DebConfirmAmt,CountDebtConfirmRelease,DebConfirmAmtRelease


In [11]:
vc(df1, 'DebtInCharge')

,DebtInCharge,counts
0,MDS,196847
1,CS,28884


In [14]:
df1[['BranchID', 'OrderNbr', 'CustID', 'CustName']].to_clipboard()

In [15]:
print(df1['OrdAmtRelease'].sum())

314483832278.0


In [17]:
df_vptt = get_ps_df(sql="select * from vptt")

In [ ]:
df_vptt = pd.read_csv('pvtt.csv')
df1 = df1.merge(df_vptt, how = 'left', left_on='State', right_on='tinh',suffixes=('_left', '_right'), validate="m:1")

In [ ]:
df_mkv_viet_tat = get_ps_df("select * from mkv_viet_tat")

NameError: name 'get_ps_df' is not defined

In [71]:
df1['vptt'].value_counts(dropna=False)

TRẠM             133349
VP TRỰC THUỘC     91011
Name: vptt, dtype: int64

In [72]:
df1['State'][df1['vptt'].isna()]

Series([], Name: State, dtype: object)

In [73]:
df1[['State','vptt']].isna()

,State,vptt
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
...,...,...
224355,False,False
224356,False,False
224357,False,False
224358,False,False


In [74]:
df1[['DateOfOrder', 'DueDate']].dtypes

DateOfOrder    datetime64[ns]
DueDate        datetime64[ns]
dtype: object

In [75]:
df1.SupName.value_counts(dropna=False)

Nguyễn Hoàng Nguyên    27171
Lâm Mạnh Quyền         23242
Nguyễn Phước Hòa       22797
Nguyễn Hoàng Mạnh      22626
Nguyễn Văn Tiến        18518
Trương Tới             18330
Nguyễn Minh Thảo       18320
Nguyễn Tùng Lâm        17833
Vũ Đình Thắng          14936
Nguyễn Thái Bảo        14428
Phan Việt Khải         11890
Lê Tấn Hải              5587
Nguyễn Chí Tâm          4456
Nguyễn Xuân Kiệt        2053
                         931
Lâm Văn Cảnh             366
Mai Thị Thanh Phúc       353
Nguyễn Thị Hằng          302
Nguyễn Thành Hiệp        171
Hoàng Trung Thành         37
Lương Trịnh Thắng          7
Trần Quang Thịnh           4
Trần Gia Vĩnh              2
Name: SupName, dtype: int64

In [76]:
groupbylist = [
    'OrderNbr', 
    'BranchID', 
    'Position', 
    'SlsperID', 
    'SupName', 
    'ASMName', 
    'RSMName', 
    'DateOfOrder', 
    'DueDate',
    'SlsperName',
    'DebtInCharge',
    'Terms', 
    'PaymentsForm',
    'TermsType',
    'Territory', 
    'State', 
    'vptt', 
    'DeliveryUnit',
    'SubChannel'
    ]

In [77]:
aggregate_dict = {
'OpeiningOrderAmt':np.sum,
'OrdAmtRelease':np.sum,
'DeliveredOrderAmt':np.sum,
'ReturnOrdAmt': np.sum,
'DebConfirmAmt': np.sum,
'DebConfirmAmtRelease': np.sum,
#DonHang8
'CountOpeningOrder':np.sum,
'CountOrdRelease': np.sum,
'DeliveredOrder':np.sum,
'CountReturnOrd': np.sum,
'CountDebtConfirm': np.sum,
'CountDebtConfirmRelease':np.sum
}

In [87]:
df2 = df1.groupby(groupbylist, dropna=False).aggregate(aggregate_dict).reset_index()

In [88]:
print(df2['OrdAmtRelease'].sum()==185823616294)

False


In [89]:
df2['DateOfOrder'].nsmallest(5)

90688   2016-12-13
90689   2016-12-14
90690   2016-12-28
0       2017-01-18
1       2017-01-18
Name: DateOfOrder, dtype: datetime64[ns]

In [90]:
df2.columns

Index(['OrderNbr', 'BranchID', 'Position', 'SlsperID', 'SupName', 'ASMName',
       'RSMName', 'DateOfOrder', 'DueDate', 'SlsperName', 'DebtInCharge',
       'Terms', 'PaymentsForm', 'TermsType', 'Territory', 'State', 'vptt',
       'DeliveryUnit', 'SubChannel', 'OpeiningOrderAmt', 'OrdAmtRelease',
       'DeliveredOrderAmt', 'ReturnOrdAmt', 'DebConfirmAmt',
       'DebConfirmAmtRelease', 'CountOpeningOrder', 'CountOrdRelease',
       'DeliveredOrder', 'CountReturnOrd', 'CountDebtConfirm',
       'CountDebtConfirmRelease'],
      dtype='object')

In [91]:
rename_dict ={
# Doanh So
'OpeiningOrderAmt': 'tiennodauky',
'OrdAmtRelease':'tienchotso',
'DeliveredOrderAmt':'tiengiaothanhcong',
'ReturnOrdAmt':'tienhuydon',
'DebConfirmAmt':'tienlenbangke',
'DebConfirmAmtRelease':'tienthuquyxacnhan',
# Don Hang
'CountOpeningOrder': 'dondauky',
'CountOrdRelease':'donchotso',
'DeliveredOrder':'dongiaothanhcong',
'CountReturnOrd':'donhuy',
'CountDebtConfirm':'donlenbangke',
'CountDebtConfirmRelease':'donthuquyxacnhan'
}
df2.rename(columns=rename_dict, inplace=True)

In [92]:
df2['tiennocongty'] = df2['tiennodauky'] + df2['tienchotso'].values - df2['tienhuydon'].values - df2['tienthuquyxacnhan'].values
# df2['tiennocongty'] = np.where(df2['chitietnocongty']>0, df2['chitietnocongty'].values, 0)
# df2['chitietdonnocongty'] = df2['dondauky'] + df2['donchotso'] - df2['donthuquyxacnhan'] - df2['donhuy']
# df2['donnocongty_old'] = df2.apply(lambda x: x['chitietdonnocongty'] if x['chitietdonnocongty']>0 else 0, axis=1)
df2['donnocongty'] = np.where(df2['tiennocongty'].values > 0, 1, 0)

df2['donchuagiao'] = df2['donchotso'] - df2['dongiaothanhcong'] - df2['donhuy']
df2['tiendonchuagiao'] = df2['tienchotso'] - df2['tiengiaothanhcong'] - df2['tienhuydon']

date_now_at_midnight = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
df2['OverDueDate'] = np.where(df2['DueDate'].values <= np.datetime64(date_now_at_midnight), True, False)
# df2['OverDueDate'].value_counts()
df2['nodenhan'] = np.where(df2['OverDueDate'].values, df2['tiennocongty'].values, 0)
df2['nochuadenhan'] = np.where(df2['OverDueDate'].values != True, df2['tiennocongty'].values, 0)

In [93]:
df2[df2['OverDueDate']].to_clipboard()

In [94]:
un(df2, 'TermsType')

array(['Cho nợ', 'Thanh Toán Ngay'], dtype=object)

In [95]:
df2[checkdup(df2, 2, ['BranchID','OrderNbr','DateOfOrder','DueDate'])]

,OrderNbr,BranchID,Position,SlsperID,SupName,ASMName,RSMName,DateOfOrder,DueDate,SlsperName,...,donhuy,donlenbangke,donthuquyxacnhan,tiennocongty,donnocongty,donchuagiao,tiendonchuagiao,OverDueDate,nodenhan,nochuadenhan


In [50]:
print_df_schema(df2)

OrderNbr VARCHAR(100) NULL,
BranchID VARCHAR(100) NULL,
Position VARCHAR(100) NULL,
SlsperID VARCHAR(100) NULL,
SupName VARCHAR(100) NULL,
ASMName VARCHAR(100) NULL,
RSMName VARCHAR(100) NULL,
DateOfOrder TIMESTAMP NULL,
DueDate TIMESTAMP NULL,
SlsperName VARCHAR(100) NULL,
DebtInCharge VARCHAR(100) NULL,
Terms VARCHAR(100) NULL,
PaymentsForm VARCHAR(100) NULL,
TermsType VARCHAR(100) NULL,
Territory VARCHAR(100) NULL,
State VARCHAR(100) NULL,
vptt VARCHAR(100) NULL,
DeliveryUnit VARCHAR(100) NULL,
SubChannel VARCHAR(100) NULL,
tiennodauky NUMERIC(24) NULL,
tienchotso NUMERIC(24) NULL,
tiengiaothanhcong NUMERIC(24) NULL,
tienhuydon NUMERIC(24) NULL,
tienlenbangke NUMERIC(24) NULL,
tienthuquyxacnhan NUMERIC(24) NULL,
dondauky BIGINT NULL,
donchotso BIGINT NULL,
dongiaothanhcong BIGINT NULL,
donhuy BIGINT NULL,
donlenbangke BIGINT NULL,
donthuquyxacnhan BIGINT NULL,
tiennocongty NUMERIC(24) NULL,
donnocongty BIGINT NULL,
donchuagiao BIGINT NULL,
tiendonchuagiao NUMERIC(24) NULL,
OverDueDa

In [4]:
postgres_conf = {"host": "171.235.26.161","port": 5432,"user": "biteam","password": "123biteam","database": "biteam"}

In [6]:
x = [x.lower() for x in df_vptt.columns]
df_vptt.columns = x

In [55]:
df3=df2.head(5)

In [7]:
insert_df_to_postgres(
postgres_conf, tbl_name="vptt",
df=df_vptt, primary_keys=['tinh'], schema="biteam")

In [98]:
insert_df_to_postgres(
postgres_conf, tbl_name="trackingdebt_2",
df=df_vptt, primary_keys=['ordernbr', 'branchid', 'dateoforder', 'duedate'], schema="biteam")